
# Rubrics Capstone API

Total score = 1 $\times$ `maximum_score`

All of these components are relatives to `maximum_score`. 


#### 1. Environment and Installations (Total 2)
**Complete at least 3 to get 2**
- [ ] Created virtual environment called "capstone-api"
- [ ] Installed Packages using conda / pip
- [ ] Export virtual environment using pip freeze

In [1]:
import flask
import requests
import gunicorn
import sqlite3

import pandas as pd

#### 2. Exploratory Data Analysis: (Total 8)

**Do at least at least 3 of the following operations to get 3**
- [ ] Datetime operation 
- [ ] Categorical operation 
- [ ] Frequencies analysis 
- [ ] Missing Value and Duplicates operation


**Do at least 2 of the following operations to get 3**
- [ ] stack/unstack
- [ ] melt 
- [ ] groupby 

**Do at least 1 of the following operations to get 2**
- [ ] Take data from joining minimum of 4 table 


In [3]:
import sqlite3
import pandas as pd

# 2. API for invoice data, with detail date and day name

conn = sqlite3.connect("data_input/chinook.db")
data = pd.read_sql_query("SELECT * FROM invoices", conn)
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['InvoiceDOW'] = data['InvoiceDate'].dt.day_name()
data.head(10)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,InvoiceDOW
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,Thursday
1,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,Friday
2,3,8,2009-01-03,Grétrystraat 63,Brussels,None,Belgium,1000,5.94,Saturday
3,4,14,2009-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91,Tuesday
4,5,23,2009-01-11,69 Salem Street,Boston,MA,USA,2113,13.86,Sunday
5,6,37,2009-01-19,Berger Straße 10,Frankfurt,None,Germany,60316,0.99,Monday
6,7,38,2009-02-01,Barbarossastraße 19,Berlin,None,Germany,10779,1.98,Sunday
7,8,40,2009-02-01,"8, Rue Hanovre",Paris,None,France,75002,1.98,Sunday
8,9,42,2009-02-02,"9, Place Louis Barthou",Bordeaux,None,France,33000,3.96,Monday
9,10,46,2009-02-03,3 Chatham Street,Dublin,Dublin,Ireland,None,5.94,Tuesday


In [9]:
# 3. to Get invoice and fitering by Country and Day name
conn = sqlite3.connect("data_input/chinook.db")
data = pd.read_sql_query("SELECT * FROM invoices", conn)
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['InvoiceDOW'] = data['InvoiceDate'].dt.day_name()
data1 = data[(data['BillingCountry'] == 'USA') & (data['InvoiceDOW'] == 'Sunday')]  
data1

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,InvoiceDOW
4,5,23,2009-01-11,69 Salem Street,Boston,MA,USA,2113,13.86,Sunday
37,38,21,2009-06-07,801 W 4th Street,Reno,NV,USA,89503,5.94,Sunday
68,69,25,2009-10-25,319 N. Frances Street,Madison,WI,USA,53703,0.99,Sunday
80,81,19,2009-12-13,1 Infinite Loop,Cupertino,CA,USA,95014,8.91,Sunday
102,103,24,2010-03-21,162 E Superior Street,Chicago,IL,USA,60611,15.86,Sunday
135,136,22,2010-08-15,120 S Orange Ave,Orlando,FL,USA,32801,5.94,Sunday
166,167,26,2011-01-02,2211 W Berry Street,Fort Worth,TX,USA,76110,0.99,Sunday
178,179,20,2011-02-20,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,8.91,Sunday
200,201,25,2011-05-29,319 N. Frances Street,Madison,WI,USA,53703,18.86,Sunday
233,234,23,2011-10-23,69 Salem Street,Boston,MA,USA,2113,5.94,Sunday


In [14]:
# 4. API to calculate the Sales Weekdays in top 5 Countries

data = pd.read_sql_query(
    '''
    SELECT Customers.Country ,invoices.Total,invoices.InvoiceDate
    FROM invoices
    LEFT JOIN customers
    ON invoices.customerId = Customers.customerId
    ''', conn,parse_dates='InvoiceDate'
)
data['InvoiceDOW'] = data['InvoiceDate'].dt.day_name()
data.groupby(['Country']).sum().sort_values(by='Total',ascending=False).head(5)

top5 = data.groupby('Country').Total.sum().sort_values(ascending=False).head().index.to_list()
top5_data = data[data['Country'].isin(top5)].copy()
dayorder = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
top5_data['InvoiceDOW'] = pd.Categorical(top5_data['InvoiceDOW'],
                                         categories=dayorder,
                                         ordered=True)
top5_data[(top5_data['InvoiceDOW'] != 'Saturday') & (top5_data['InvoiceDOW'] != 'Sunday')].\
pivot_table(index='InvoiceDOW',
               columns='Country',
              values='Total',
              aggfunc='sum')


Country,Brazil,Canada,France,Germany,USA
InvoiceDOW,,,,,
Monday,19.80,43.56,25.75,64.35,19.80
Tuesday,1.98,22.78,1.98,9.90,123.77
Wednesday,26.73,90.09,11.88,24.75,41.62
Thursday,45.56,43.58,47.52,3.96,77.27
Friday,16.83,58.41,1.98,8.91,87.18


In [16]:
# 5. API to calculate the top Genre in Country ('USA', 'Canada', 'France', 'Brazil', 'Germany')

genre = pd.read_sql_query(
 '''
 SELECT 
 BillingCountry AS Country, genres.Name AS Genre , tracks.Unitprice, invoices.Total
 FROM invoices
 LEFT JOIN invoice_items ON invoices.InvoiceId = invoice_items.InvoiceId
 LEFT JOIN tracks ON invoice_items.TrackId = tracks.TrackId 
 LEFT JOIN genres ON tracks.genreId = genres.genreId
 WHERE BillingCountry IN ('USA', 'Canada', 'France', 'Brazil', 'Germany')
 ''',conn)
genre.head()

,Country,Genre,UnitPrice,Total
0,Germany,Rock,0.99,1.98
1,Germany,Rock,0.99,1.98
2,Canada,Rock,0.99,8.91
3,Canada,Rock,0.99,8.91
4,Canada,Rock,0.99,8.91


In [17]:
# 6. API to calculate and select the Genre music 

genre = pd.read_sql_query(
 '''
 SELECT 
 BillingCountry AS Country, genres.Name AS Genre , tracks.Unitprice, invoices.Total
 FROM invoices
 LEFT JOIN invoice_items ON invoices.InvoiceId = invoice_items.InvoiceId
 LEFT JOIN tracks ON invoice_items.TrackId = tracks.TrackId 
 LEFT JOIN genres ON tracks.genreId = genres.genreId
 WHERE BillingCountry IN ('USA', 'Canada', 'France', 'Brazil', 'Germany')
 ''',conn)
genre = genre[(genre['Genre'] == 'Rock')]
genre.head()



,Country,Genre,UnitPrice,Total
0,Germany,Rock,0.99,1.98
1,Germany,Rock,0.99,1.98
2,Canada,Rock,0.99,8.91
3,Canada,Rock,0.99,8.91
4,Canada,Rock,0.99,8.91


#### 3. Deployment (Total 3)

**Complete at least 1 to get 1 :**
- [ ] Deploy to Heroku using Github 
- [ ] Deploy to Heroku using Heroku Git 

**Complete at least 2 to get 2 :**
- [ ] Create 2 or more static endpoints
- [ ] Create 1 or more dynamic endpoint

In [ ]:
##3

#### 4. Documentation (Total 3)
**Complete at least 2 to get 3 points :**
- [ ] Create documentation as markdown in Github 
- [ ] Create documentation page in `<your_api_url>/docs` (may require a little HTML knowledge)
___